In [ ]:
import sys
sys.path.append('VDAM2022') # Comment out for use
from data_processing import ConnDF, default_data
from analysis import SummaryDF, GraphDF

In [ ]:
import analysis, data_processing
from importlib import reload
reload(analysis)
reload(data_processing)
from analysis import SummaryDF, GraphDF
from data_processing import ConnDF

In [ ]:
data = default_data()

In [ ]:
random_data = default_data()

In [ ]:
random_data.conn_master = random_data.randomise(random_data.conn_master)

In [ ]:
random_data.df_as_graph(package='igraph', parallel_edges=False)

In [ ]:
data.df_as_graph(package='igraph', parallel_edges=False)

In [ ]:
data_igraph = data.graph

In [ ]:
random_data_igraph = random_data.graph

In [ ]:
import leidenalg
modularity_profile = leidenalg.find_partition(data_igraph, leidenalg.ModularityVertexPartition, weights='weight', n_iterations=-1)

In [ ]:
random_modularity_profile = leidenalg.find_partition(random_data_igraph, leidenalg.ModularityVertexPartition, weights='weight', n_iterations=-1)

In [ ]:
modularity_profile

In [ ]:
data.annotate_top_roi()

In [ ]:
gs = GraphDF(graph=data_igraph, profile=modularity_profile, conndf=data)
gs.vertex_df = gs.vertex_df[gs.vertex_df['class'] != 'TBD'] #Remove TBD nodes

import pandas as pd
gs.vertex_df.rename({'_nx_name': 'bodyId'}, inplace=True, axis=1)
gs.vertex_df = pd.merge(gs.vertex_df, data.neuron_master[['bodyId','mainRoi']], on='bodyId', how='left')

In [ ]:
gs.vertex_df.to_csv('with_comm.csv')

In [ ]:
rand_gs = GraphDF(graph=random_data_igraph, profile=random_modularity_profile, conndf=random_data)
rand_gs.vertex_df = rand_gs.vertex_df[rand_gs.vertex_df['class'] != 'TBD'] #Remove TBD nodes

import pandas as pd
rand_gs.vertex_df.rename({'_nx_name': 'bodyId'}, inplace=True, axis=1)
rand_gs.vertex_df = pd.merge(rand_gs.vertex_df, data.neuron_master[['bodyId','mainRoi']], on='bodyId', how='left')

In [ ]:
# Get the top counts per ROI
counts_roi

In [ ]:
cmap_dict

In [ ]:
order = ['LTct', 'HTct(UTct-T3)(L)', 'ANm', 'LegNp(T3)(R)', 'LegNp(T1)(R)','LegNp(T1)(L)', 'LegNp(T2)(R)','LegNp(T3)(L)','LegNp(T2)(L)','Ov(R)','mVAC(T1)(L)']
cols = [list(cmap_dict[x]) for x in order]

In [ ]:
import igraph as ig

rand_manual_partition = rand_gs.get_manual_partition(8)
#manual_partition.graph.es['color'] = ['red' if x < 0 else 'blue' for x in list(ged['signed_weight'])]
#pal = ig.ClusterColoringPalette(rand_gs.vertex_df.comm_attr.nunique())
randplot, randpal = rand_gs.plot_by_class(
    rand_manual_partition, 
    layout='fruchterman_reingold', 
    label_nodes=False, 
    edge_width=0.05, 
    edge_arrow_size=0.1, 
    edge_size=0.05,
    edge_color='black') ## Can suppress e/i

In [ ]:
randplot

In [ ]:
cols

In [ ]:
manual_partition = gs.get_manual_partition(50)
#manual_partition.graph.es['color'] = ['red' if x < 0 else 'blue' for x in list(ged['signed_weight'])]
pal = ig.PrecalculatedPalette(cols)
mplot = gs.plot_by_class(
    manual_partition, 
    layout='fruchterman_reingold', 
    label_nodes=False, 
    edge_width=0.05, 
    edge_arrow_size=0.1, 
    edge_size=0.05,
    edge_color='black',
    palette=pal) ## Can suppress e/i

In [ ]:
pal

In [ ]:
mplot

In [ ]:
rand_gs.vertex_df.comm_attr.value_counts()

In [ ]:
gs.vertex_df['comm_attr'].value_counts()

In [ ]:
rand_gs.vertex_df['comm_attr'].value_counts()

In [ ]:
gs.vertex_df['comm_attr'].value_counts()*100/gs.vertex_df['comm_attr'].value_counts().sum()

In [ ]:
mat = 
k = mat.shape[0] # A graph with n nodes has n eigenvalues
evals, evecs = scipy.sparse.linalg.eigs(mat.toarray(), k=k) # Eigenvalue spectrum
plt.figure(figsize=(10,10))
# complex plane
# plot the complex numbers
plt.scatter(evals.real, evals.imag, c='blue')
#plt.scatter(evals.real*1.5, evals.imag*1.5, c='blue') #firing rate slope = 1.5
plt.ylabel('Imaginary')
plt.xlabel('Real')
plt.axvline(x=1, c='red')
plt.axvline(x=-1, c='red')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
k = list(gs.vertex_df.comm_attr.value_counts().index)
v = list(gs.vertex_df.comm_attr.value_counts().values)
colors = [mpal.get(i) for i in k]
plt.bar(k,v, color=colors, edgecolor='black', width=0.8)
plt.xticks(ticks=k)
plt.ylabel('Neuron count')
plt.xlabel('Community index')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch


# Create legend handles and labels from the color mapping dictionary
legend_handles = [Patch(facecolor=color) for color in [pal.get(i) for i in k]]
legend_labels = k

# Create the legend
plt.legend(legend_handles, legend_labels, title='Community index')

# Show the plot
plt.show()


In [ ]:
df = data.neuron_master
df[df['class']=='Motor neuron']['mainRoi'].value_counts()

In [ ]:
df.roiInfo[3]

In [ ]:
t = df[df['class']=='Motor neuron'] 
t[t['roiInfo'].str.startswith('{"LegNp(T')].roiInfo.str[:15].unique()

In [ ]:
gs.cmap

In [ ]:
gs.vertex_df['tw'] = gs.vertex_df['downstream'] + gs.vertex_df['upstream']

In [ ]:
gs.vertex_df.comm_attr.value_counts()

In [ ]:
merger = list()
for c in range(11):
    merger += list(gs.vertex_df.query(f"comm_attr == {c}").sort_values(by='wc_strength', ascending=False).head(30)._nx_name)

In [ ]:
df = gs.vertex_df.query(f"_nx_name.isin({merger})")

In [ ]:
import matplotlib.patches as mpatches

top = 30
merger = list()
for c in range(11):
    merger += list(gs.vertex_df.query(f"comm_attr == {c}").sort_values(by='wc_strength', ascending=False).head(top)._nx_name)
df = gs.vertex_df.query(f"_nx_name.isin({merger})")
counts = df.groupby(['comm_attr', 'class']).size().unstack()
counts.plot(kind='bar', stacked=True, color=[gs.cmap.get(category) for category in counts.columns], width=0.8, edgecolor='black')
plt.xticks(rotation=0)
plt.ylabel('Neuron count')
plt.xlabel('Community index')

patches = [mpatches.Patch(color=[r,g,b], label=category)
           for category, (r, g, b) in gs.cmap.items()]
for handle in patches:
    handle.set_edgecolor('black')
    handle.set_linewidth(1.0)
           
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5), handles=patches) 

In [ ]:
import matplotlib.pyplot as plt
#gs = GraphDF(graph=random_data_igraph, profile=random_modularity_profile, conndf=random_data)
%matplotlib inline
gs.vertex_df = gs.vertex_df[gs.vertex_df['class'] != 'TBD'] #Remove TBD nodes
import matplotlib.patches as mpatches

g_instance = gs

counts = g_instance.vertex_df.groupby(['comm_attr', 'class']).size().unstack()
counts = counts.fillna(0).apply(lambda x: x/counts.fillna(0).sum(axis=1), axis=0)
counts.plot(kind='bar', stacked=True, color=[gs.cmap.get(category) for category in counts.columns], width=0.8, edgecolor='black')

plt.ylabel('Neuron proportion')
plt.xlabel('Community index')


patches = [mpatches.Patch(color=[r,g,b], label=category)
           for category, (r, g, b) in gs.cmap.items()]
for handle in patches:
    handle.set_edgecolor('black')
    handle.set_linewidth(1.0)
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5), handles=patches, title='Neuron Class') 


# Chi-square on class
no_class = data.neuron_master[data.neuron_master['class'].isna()]['bodyId'].to_list()
counts_class = g_instance.vertex_df[~g_instance.vertex_df['bodyId'].isin(no_class)].groupby(['comm_attr', 'class']).size().unstack()
counts_class = counts_class.fillna(0)
counts_class.drop('Glia', axis=1, inplace=True)

import numpy as np
import scipy
chi2, p, dof, expected = scipy.stats.chi2_contingency(counts_class.loc[[3,4,5,6,7,8],:].to_numpy())
print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}") 
print(f"N: {counts_class.sum().sum()}")

In [ ]:
counts_class

In [ ]:
data.neuron_master.query("bodyId == 13809").roiInfo.to_csv()

In [ ]:
data.annotate_top_roi()

In [ ]:
counts.fillna(0)

In [ ]:
counts.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
# Generate some example categories (replace with your actual categories)
categories = np.arange(30)

# Generate a list of 30 evenly spaced values between 0 and 1
values1 = np.linspace(0, 1, 12)
values2 = np.linspace(0, 1, 9)
values3 = np.linspace(0, 1, 8)

# Create a list of colors with distinct values for each category
colors1 = plt.cm.get_cmap('Set3')(values1)
colors2 = plt.cm.get_cmap('Pastel1')(values2)
colors3 = plt.cm.get_cmap('Dark2')(values3)
colors = np.concatenate((colors1, colors2, colors3))

shuffled_indices = np.random.permutation(len(colors))
# Reorder the colors array using shuffled indices
shuffled_colors = colors[shuffled_indices]

# Create a custom colormap using LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, len(colors))

In [ ]:
data.neuron_master.mainRoi.unique()

In [ ]:
gs.vertex_df.comm_attr.value_counts().sum()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

g_instance = gs

g_instance.vertex_df = g_instance.vertex_df[g_instance.vertex_df['class'] != 'TBD'] #Remove TBD nodes
import matplotlib.patches as mpatches

counts = g_instance.vertex_df.groupby(['comm_attr', 'mainRoi']).size().unstack()
#rel_counts = counts.fillna(0).apply(lambda x: x/counts.sum(axis=1))

## CMAP
# Generate some example categories (replace with your actual categories)
categories = np.arange(30)

# Generate a list of 30 evenly spaced values between 0 and 1
values1 = np.linspace(0, 1, 12)
values2 = np.linspace(0, 1, 9)
values3 = np.linspace(0, 1, 8)

# Create a list of colors with distinct values for each category
colors1 = plt.cm.get_cmap('Set3')(values1)
colors2 = plt.cm.get_cmap('Set1')(values2)
colors3 = plt.cm.get_cmap('Dark2')(values3)
colors = np.concatenate((colors1, colors2, colors3))
shuffled_indices = np.random.RandomState(seed=13809).permutation(len(colors))
# Reorder the colors array using shuffled indices
colors = colors[shuffled_indices]
# Create a custom colormap using LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, len(colors))
cmap_dict = dict(zip(counts.columns.to_list(), colors))

counts = counts.fillna(0).apply(lambda x: x/counts.fillna(0).sum(axis=1), axis=0)
counts.plot(kind='bar', stacked=True, width=0.8, edgecolor='black', color=cmap_dict)

plt.ylabel('Neuron proportion')
plt.xlabel('Community index')
           
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=4, title='Main ROI') 

# Chi-square on class
no_class = data.neuron_master[data.neuron_master['class'].isna()]['bodyId'].to_list()
counts_roi = g_instance.vertex_df[~g_instance.vertex_df['bodyId'].isin(no_class)].groupby(['comm_attr', 'mainRoi']).size().unstack()
counts_roi = counts_roi.fillna(0)
counts_roi.rename(columns={'':'None'}, inplace=True)
counts_roi.drop('None', inplace=True, axis=1)

import numpy as np
import scipy
chi2, p, dof, expected = scipy.stats.chi2_contingency(counts_roi.to_numpy())
print(f"chi2 statistic:     {chi2:.2f}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}") 
print(f"N: {counts_class.sum().sum()}")

In [ ]:
counts_roi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

g_instance = gs

g_instance.vertex_df = g_instance.vertex_df[g_instance.vertex_df['class'] != 'TBD'] #Remove TBD nodes
import matplotlib.patches as mpatches

counts = g_instance.vertex_df[g_instance.vertex_df['class'] == 'Motor neuron'].groupby(['comm_attr', 'mainRoi']).size().unstack()
#rel_counts = counts.fillna(0).apply(lambda x: x/counts.sum(axis=1))
counts.loc[7,:] = pd.Series(0)
counts.loc[9,:] = pd.Series(0)
counts.loc[10,:] = pd.Series(0)
counts.sort_index(inplace=True)
counts.plot(kind='bar', stacked=True, width=0.8, edgecolor='black', color=cmap_dict)



plt.ylabel('Motor Neuron count')
plt.xlabel('Community index')
plt.xticks(rotation=0)
           
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2, title='Main ROI') 

In [ ]:
counts['ANm']

In [ ]:
counts.fillna(0)

In [ ]:
counts.loc[7,:] = pd.Series(None)
counts.loc[9,:] = pd.Series(None)
counts.loc[10,:] = pd.Series(None)

In [ ]:
counts

In [ ]:
no_class = data.neuron_master[data.neuron_master['class'].isna()]['bodyId'].to_list()

In [ ]:
gs.vertex_df['_nx_name'].isin(no_class)

In [ ]:
# Chi-square on class
g_instance = gs
counts_class = g_instance.vertex_df[~g_instance.vertex_df['bodyId'].isin(no_class)].groupby(['comm_attr', 'class']).size().unstack()
counts_class = counts_class.fillna(0)
counts_class = counts_class.drop('None', axis=1)

In [ ]:
counts_class

In [ ]:
import numpy as np
import scipy
chi2, p, dof, expected = scipy.stats.chi2_contingency(counts_class.to_numpy())
print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")
print(f"N: {counts_class.sum().sum()}")

In [ ]:
print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")
print(f"N: {counts_class.sum().sum()}")

In [ ]:
counts

In [ ]:
comm1_df = gs.vertex_df.query("comm_attr == 1")
comm1_df[comm1_df['class'] == 'Motor neuron'][['_nx_name', 'type', 'somaSide', 'somaNeuromere', 'mainRoi']]

In [ ]:
r = data.neuron_master.query("bodyId == 10462")['roiInfo']

In [ ]:
data.neuron_master['mainRoi']

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 1")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 1")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()
mn_1 = list(comm_x_df[comm_x_df['class'] == 'Motor neuron']['bodyId'])

reload(analysis)
import analysis
from analysis import ClusterDF

viz = ClusterDF(data)
viz.draw_morphologies(bodyId_list=mn_1)

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 6")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 2")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 4")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 5")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 8")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm1_df = gs.vertex_df.query("comm_attr == 1")
vc1 = comm1_df[comm1_df['class'] == 'Motor neuron'].type.value_counts()

In [ ]:
comm6_df = gs.vertex_df.query("comm_attr == 6")
vc6 = comm6_df[comm6_df['class'] == 'Motor neuron'].type.value_counts()

In [ ]:
import pandas as pd
merged_series = pd.merge(vc1, vc6, left_index=True, right_index=True)

In [ ]:
merged_series.rename({'type_x':'Community 1', 'type_y':'Community 6'}).to_csv('seres')

In [ ]:
vc_dict = dict()
for i in range(11):
    comm_df = gs.vertex_df.query(f"comm_attr == {i}")
    vcx = comm_df[comm_df['class'] == 'Motor neuron'].exitNerve.value_counts()
    vc_dict[i] = vcx
    print(f'Community {i}')
    print(vcx)
    print('\n')
merged_series = pd.concat([x for x in vc_dict.values()], axis=1, join='outer') 
merged_series = merged_series.fillna(0)
merged_series.columns = [f'Community {i}' for i in range(11)]
merged_series = merged_series.astype(int)

In [ ]:
d = merged_series[[f'Community {i}' for i in [1,6,2,4,5,8]]]

In [ ]:
merged_series

In [ ]:
merged_series = pd.concat([x for x in vc_dict.values()], axis=1, join='outer') 
merged_series = merged_series.fillna(0)
merged_series.columns = [f'Community {i}' for i in range(11)]
merged_series = merged_series.astype(int)
merged_series_reindexed = merged_series.reset_index()
merged_series.drop(axis=0)
df = merged_series_reindexed[merged_series_reindexed['index'].str.contains('Ti') | merged_series_reindexed['index'].str.contains('ti')]
df[df != 0]

In [ ]:
merged_series

In [ ]:
df = merged_series

# Set the same color scheme for all pie charts
colors = plt.cm.Set3.colors

for column in df.columns:
    community_data = df[column]
    total_count = community_data.sum()
    percentages = (community_data / total_count) * 100

    # Filter out categories with 0 counts
    nonzero_percentages = percentages[percentages > 0]

    # Create the pie chart
    plt.figure()
    patches, texts, autotexts = plt.pie(nonzero_percentages, labels=nonzero_percentages.index,
                                        autopct='%1.1f%%', colors=colors, textprops={'fontsize': 7})

    # Move percentage text closer to the edge
    for autotext, patch, label in zip(autotexts, patches, nonzero_percentages.index):
        angle = np.deg2rad((patch.theta2 - patch.theta1) / 2. + patch.theta1)
        x = np.cos(angle)
        y = np.sin(angle)
        radius = 0.7 * patch.r  # Adjust the radius as needed (0.7 is just an example)
        
        if label.endswith('_R'):
            x_text = radius * 1.2 * np.cos(angle)
            y_text = radius * 1.2 * np.sin(angle)
        elif label.endswith('_L'):
            x_text = radius * 0.8 * np.cos(angle)
            y_text = radius * 0.8 * np.sin(angle)
        else:
            x_text = radius * np.cos(angle)
            y_text = radius * np.sin(angle)
        
        autotext.set_position((x_text, y_text))

    plt.title(f'')
    plt.axis('equal')
    plt.show()

In [ ]:
merged_series.T.plot(kind='bar', stacked=True)
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2, title='Exit Nerve') 

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,8))
sns.heatmap(merged_series, cmap='viridis')
plt.ylabel('Exit Nerve')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
gs.vertex_df = gs.vertex_df[gs.vertex_df['class'] != 'TBD'] #Remove TBD nodes
import matplotlib.patches as mpatches

counts = gs.vertex_df.groupby(['comm_attr', 'mainRoi']).size().unstack()
rel_counts = counts.fillna(0).apply(lambda x: x/counts.sum(axis=1))
rel_counts.plot(kind='bar', stacked=True, width=0.8, edgecolor='black')

plt.ylabel('Relative Neuron Fraction')
plt.xlabel('Community index')
           
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5)) 

In [ ]:
MDN_L = [13438, 14523]
MDN_R = [13809, 14419]
LUL130 = [14084, 13323, 13574, 13293]
LBL40 = [10994, 11493]
LUM9 = [16903, 18491]
ALL = MDN_L + MDN_R + LUL130 + LBL40 + LUM9

In [ ]:
cdf = gs.vertex_df.query("comm_attr == 2")
cdf = cdf[cdf['class'] == 'Motor neuron']
cdf.type.value_counts()

In [ ]:
cdf.exitNerve.unique()

In [ ]:
cdf = gs.vertex_df.query("comm_attr == 4")
cdf = cdf[cdf['class'] == 'Motor neuron']
cdf.type.value_counts()
cdf.exitNerve.unique()

In [ ]:
cdf = gs.vertex_df.query("comm_attr == 5")
cdf = cdf[cdf['class'] == 'Motor neuron']
cdf.type.value_counts()
cdf.exitNerve.unique()

In [ ]:
cdf = gs.vertex_df.query("comm_attr == 6")
cdf = cdf[cdf['class'] == 'Motor neuron']
cdf.type.value_counts()
cdf.exitNerve.unique()

In [ ]:
cdf = gs.vertex_df.query("comm_attr == 8")
cdf = cdf[cdf['class'] == 'Motor neuron']
cdf.type.value_counts()
cdf.exitNerve.unique()

In [ ]:
tps = cdf.type.value_counts().index
storage = []
for i in tps:
    if i not in storage:
        print(f"From {i}, to:")
        print(cdf.query(f'type == "{i}"').exitNerve.unique())
    storage.append(i)

In [ ]:
comm_x_df[comm_x_df['class'] == 'Motor neuron'].query('exitNerve == "AbN1_R"')

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 7")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()
comm_x_df[comm_x_df['class'] == 'Motor neuron'].query('exitNerve == "AbN1_L"')

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 7")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df = gs.vertex_df.query("comm_attr == 3")
comm_x_df[comm_x_df['class'] == 'Motor neuron'].exitNerve.value_counts()

In [ ]:
comm_x_df

In [ ]:
qdf = gs.vertex_df.query(f'bodyId.isin({ALL})')

In [ ]:
qdf.groupby('type').sum()

In [ ]:
qdf.query('comm_attr==6')

In [ ]:
rdf = rand_gs.vertex_df.query(f'_nx_name.isin({ALL})')

In [ ]:
rdf[['_nx_name', 'type','comm_attr']]

In [ ]:
qdf[['_nx_name', 'type','comm_attr']]

In [ ]:
data.neuron_master.query('bodyId==11493').somaSide

In [ ]:
gs.vertex_df

In [ ]:
import navis
from analysis import ClusterDF
viz = ClusterDF(data)
VNC_roi_meshes = viz.get_vnc_roi_meshes()

fig = navis.plot3d(
    x=VNC_roi_meshes,
    inline=False,
    width=500,
    height=800,
    legend_group={13809:'MDN (13809)',14419:'MDN (14419)',13438:'MDN (13438)',14523:'MDN (14523)',11493:'LBL40 (11493)', 10994:'LBL40 (10994)', 13293:'LUL130 (13293)', 13574:'LUL130 (13574)', 14084:'LUL130 (14084)', 13323:'LUL130 (13323)'},
    backend='plotly'
)
fig.show(renderer='notebook')

In [ ]:
reload(analysis)
import analysis
from analysis import ClusterDF

viz = ClusterDF(data)
viz.draw_morphologies(bodyId_list=[13438, 14523, 14084, 13323, 10994])

In [ ]:
gs.vertex_df['comm_attr'].value_counts()

In [ ]:
sample_d = eval(gs.vertex_df.roiInfo[0])

In [ ]:
data.annotate_top_roi()

In [ ]:
import matplotlib.patches as mpatches
gs.vertex_df['mainRoi'] = data.neuron_master.mainRoi
counts = gs.vertex_df.groupby(['comm_attr', 'mainRoi']).size().unstack()
counts.plot(kind='bar', stacked=True)
plt.xticks(rotation=0)
plt.ylabel('Neuron count')
plt.xlabel('Community index')
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5)) 

In [ ]:
counts

In [ ]:
grouped_counts.fillna(0)

In [ ]:
table = grouped_counts.fillna(0)

In [ ]:
table

In [ ]:
# chisquare the above. Is there an association between comm_attr and mainRoi?
import scipy
df = rand_gs.vertex_df
# Group by 'Cluster' and 'Category', and calculate value counts
grouped_counts = df.groupby(['comm_attr', 'class']).size().unstack()
table = grouped_counts.fillna(0)
chi2, p_value = scipy.stats.chisquare(grouped_counts.fillna(0).T, axis=1)

In [ ]:
table

In [ ]:
chi2, p, dof, expected = scipy.stats.chi2_contingency(table.to_numpy())

In [ ]:
print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")


In [ ]:
# chisquare the above. Is there an association between comm_attr and mainRoi?
import scipy
df = gs.vertex_df
# Group by 'Cluster' and 'Category', and calculate value counts
grouped_counts = df.groupby(['comm_attr', 'mainRoi']).size().unstack()
table = grouped_counts.fillna(0)
chi2, p, dof, expected = scipy.stats.chi2_contingency(table.to_numpy())

In [ ]:
table.T

In [ ]:
import numpy as np
gs.vertex_df.replace('None', np.nan).isna().sum().sort_values(ascending=True)

In [ ]:
gs.vertex_df.roiInfo[0]

In [ ]:
print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")


In [ ]:
# MDNS 13809, 14419, 13438, 14523

In [ ]:
gs.vertex_df.query("_nx_name.isin([13809, 14419, 13438, 14523])")

In [ ]:
memory = gs.vertex_df

In [ ]:
signed_igraph = data.df_as_graph(make_ei=True)

In [ ]:
edge_ei_frame = data.graph.get_edge_dataframe()

In [ ]:
edge_ei_frame

In [ ]:
['red' if x < 0 else 'green' for x in edge_ei_frame.signed_weight]

In [ ]:
gs.graph.get_edge_dataframe()

In [ ]:
ged = gs.graph.get_edge_dataframe()

In [ ]:
ged

In [ ]:
import analysis, data_processing
from importlib import reload
reload(analysis)
reload(data_processing)
from analysis import SummaryDF, GraphDF
from data_processing import ConnDF

In [ ]:
data.df_as_graph(make_ei=False)

In [ ]:
norm_igraph = data.graph

In [ ]:
gs = GraphDF(graph=norm_igraph, profile=modularity_profile, conndf=data)
%matplotlib inline
gs.vertex_df = gs.vertex_df[gs.vertex_df['class'] != 'TBD'] #Remove TBD nodes

In [ ]:
gs.vertex_df = gs.vertex_df.query('comm_attr.isin([1,6])')

In [ ]:
gs.vertex_df[gs.vertex_df['class'] == 'Motor neuron'].type.unique()

In [ ]:
manual_partition = gs.get_manual_partition(500)
manual_partition.graph.es['color'] = ['red' if x < 0 else 'blue' for x in list(ged['signed_weight'])]
mplot, mpal = gs.plot_by_class(
    manual_partition, 
    layout='fruchterman_reingold', 
    label_nodes=False, 
    edge_width=0.05, 
    edge_arrow_size=0.05, 
    edge_size=0.05,
    edge_color='black') ## Can suppress e/i

In [ ]:
mplot

In [ ]:
mplot2, mpal2 = gs.plot_by_class(n=75, layout='fruchterman_reingold', edge_color='black', edge_width=0.1, edge_arrow_size=0.1, edge_size=0.1)

In [ ]:
mplot2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = gs.vertex_df

# Group by 'Cluster' and 'Category', and calculate value counts
grouped_counts = df.groupby(['comm_attr', 'mainRoi']).size().unstack()

# Get unique clusters
clusters = grouped_counts.index


# Calculate the number of rows and columns for subplots
num_clusters = len(clusters)
num_cols = min(num_clusters, 4)
num_rows = (num_clusters - 1) // num_cols + 1

# Create the figure and subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12*1.5, 1.4*4 * num_rows))
fig.subplots_adjust(hspace=0.4)

# Flatten the axes array if necessary
if num_clusters > 1:
    axes = axes.flatten()
else:
    axes = [axes]


# Plot pie chart for each cluster
for i, cluster in enumerate(clusters):
    counts = grouped_counts.loc[cluster].fillna(0)
    categories = counts.index

    # Create a figure and axes for each cluster
    ax = axes[i]

    # Create a pie chart
    
    wedges, text, _ = ax.pie(counts, startangle=90, autopct='%1.1f%%', pctdistance=1.2)

    # Set colors for the wedges based on the category
    colors = [gs.cmap.get(category, 'gray') for category in categories]

    # Customize the percentage labels
    label_values = counts.values
    label_percentages = label_values / sum(label_values) * 100

    for i, label in enumerate(_):
        if label_percentages[i] > 0.5:  # Set a threshold for label display
            label.set_text(f'{label_percentages[i]:.1f}%')
        else:
            label.set_text('')


    # Add title
    ax.set_title(f'Community {cluster}')

    # Equal aspect ratio ensures a circular pie chart
    ax.axis('equal')

# Hide unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')
    # Show the plot
plt.show()

In [ ]:
df = gs.vertex_df
df[df['class'].isna()].comm_attr.value_counts()

In [ ]:
df.comm_attr.value_counts()

In [ ]:
(1/846)*100

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = gs.vertex_df

# Group by 'Cluster' and 'Category', and calculate value counts
grouped_counts = df.groupby(['comm_attr', 'class']).size().unstack()

# Get unique clusters
clusters = grouped_counts.index

# Plot pie chart for each cluster
for cluster in clusters:
    counts = grouped_counts.loc[cluster].fillna(0)
    categories = counts.index

    # Create a figure and axes for each cluster
    fig, ax = plt.subplots()

    # Create a pie chart
    wedges, autotexts = ax.pie(counts, startangle=90, pctdistance=0.85)

    # Set colors for the wedges based on the category
    colors = [gs.cmap.get(category, 'gray') for category in categories]
    for i, wedge in enumerate(wedges):
        wedge.set_facecolor(colors[i])

    # Customize the percentage labels
    label_values = counts.values
    label_percentages = label_values / sum(label_values) * 100

    for i, label in enumerate(autotexts):
        if label_percentages[i] > 0.4:  # Set a threshold for label display
            label.set_text(f'{label_percentages[i]:.1f}%')
        else:
            label.set_text('')
    
    # Add title
    ax.set_title(f'Community {cluster}')

    # Equal aspect ratio ensures a circular pie chart
    ax.axis('equal')

    # Show the plot
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch


# Create legend handles and labels from the color mapping dictionary
legend_handles = [Patch(facecolor=color) for color in gs.cmap.values()]
legend_labels = list(gs.cmap.keys())

# Create the legend
plt.legend(legend_handles, legend_labels, title='Neuron class')

# Show the plot
plt.show()


In [ ]:
gs.vertex_df['mainRoi'] = data.neuron_master.mainRoi

In [ ]:
for s in 

In [ ]:
gs.vertex_df[['_nx_name', 'roiInfo']]

In [ ]:
gs.vertex_df.roiInfo.unique()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def show_rgb_color(rgb):
    r, g, b = rgb

    fig, ax = plt.subplots()
    rect = patches.Rectangle((0, 0), 1, 1, facecolor=(r, g, b))
    ax.add_patch(rect)
    ax.axis('off')
    plt.show()

def show_rgba_color(rgba):
    r, g, b, a = rgba

    fig, ax = plt.subplots()
    rect = patches.Rectangle((0, 0), 1, 1, facecolor=(r, g, b, a))
    ax.add_patch(rect)
    ax.axis('off')
    plt.show()
show_rgb_color((0.0, 0.10980392156862745, 0.4980392156862745))

In [ ]:
summary = SummaryDF(data)

In [ ]:
summary.count_summary()

In [ ]:
summary.conndf.neuron_master['class'].value_counts()

In [ ]:
data.neuron_master.somaNeuromere.unique()

In [ ]:
summary.class_summary()

In [ ]:
summary.MN_summary()

In [ ]:
15 + 15 + 3 + 3 + 9 + 9 + x41 + 38 + 58 + 56 + 61 + 54 + 11 + 11 + 11 + 12 + 14 + 13

In [ ]:
data.neuron_master.pre.min()

In [ ]:
data.neuron_master.pre.max()

In [ ]:
data.neuron_master.post.min()

In [ ]:
data.neuron_master.post.max()

In [ ]:
data.neuron_master.query(f"pre == {data.neuron_master.pre.min()}").head()

In [ ]:
data.neuron_master.query(f"pre == {data.neuron_master.pre.min()}")["class"].value_counts()

In [ ]:
data.neuron_master.query(f"pre == {data.neuron_master.pre.max()}").head()

In [ ]:
data.conn_master.query("bodyId_pre == 10016")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(data.conn_master.weight, bins=100000)
plt.xscale('log')
plt.yscale('log')

In [ ]:
data.conn_master.weight

In [ ]:
import numpy as np
connection_strengths = data.conn_master.weight
bins = np.logspace(np.log10(min(connection_strengths)), np.log10(max(connection_strengths)), num=10)

# Computing the histogram
hist, bin_edges = np.histogram(connection_strengths, bins=bins)

In [ ]:
bins